In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004931449890136719
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 13.395816388535211
200000 20.257701848875072
300000 19.797774904982102
400000 19.74235242488706
500000 19.662134665700027
600000 20.39174352208299
700000 19.430315299023853
800000 19.355467771433858
900000 19.34255245272395
1000000 17.051280763140227
1100000 9.141540566206466
1200000 19.08186726564171
1300000 19.198974098810506
1400000 18.85663235505833
1500000 19.15290794337369
1600000 20.320445926435262
1700000 19.040859614034687
1800000 19.35965853355116
1900000 19.476665870737172
2000000 19.884113055557084
2100000 5.236182386775879
2200000 22.288123682693907
2300000 24.109202797832047
2400000 23.677566165488795
2500000 23.691989708934837
2600000 24.23538978751591
2700000 22.929999539242832
2800000 22.532859087614955
2900000 22.827307324824087
3000000 2

29400000 17.03705718869698
29500000 17.432173092725886
29600000 16.2703271581513
29700000 11.690106720190677
29800000 11.489148409335137
29900000 16.294338403456774
30000000 17.006023972780373
30100000 16.71314846054006
30200000 18.240754368117113
30300000 17.859185921181947
30400000 18.65151507649841
30500000 12.044246141251007
30600000 14.449280638273493
30700000 18.395335652145135
30800000 18.49138388356354
30900000 18.50875051471024
31000000 9.664248038640133
31100000 16.370881895907335
31200000 18.869437605727057
31300000 18.395552703160178
31400000 18.554366248492688
31500000 17.22676869840914
31600000 16.89241780210511
31700000 17.071334230842417
31800000 5.593786569273871
31900000 17.45046984414035
32000000 16.592585659223275
32100000 17.212087793565413
32200000 17.356362982626838
32300000 18.16041499616096
32400000 18.22179719104124
32500000 16.040003441413887
32600000 10.507677617304976
32700000 17.855369954112405
32800000 18.253083721029817
32900000 18.308577494620998
330000

59100000 15.488451113684215
59200000 16.405041348868092
59300000 15.965585835975695
59400000 16.22744165299292
59500000 16.731169516306664
59600000 16.361764814806854
59700000 16.275234834819493
59800000 16.047806925126313
59900000 8.846523281258321
60000000 13.366917334482736
60100000 14.862186413776195
60200000 14.859917256614962
60300000 14.547315091670512
60400000 13.368341034871651
60500000 6.978901785322017
60600000 14.599580319293949
60700000 14.348234958648337
60800000 13.923182191461057
60900000 14.274591831759773
61000000 8.564984225918423
61100000 14.441609881843773
61200000 15.791388158158814
61300000 15.82033965550367
61400000 16.331418261603815
61500000 16.722115733078745
61600000 16.208839334414385
61700000 15.388287839559263
61800000 15.936062442727629
61900000 15.53565133448899
62000000 6.828156535605119
62100000 14.27252612531148
62200000 13.848872671162232
62300000 13.873344234639548
62400000 13.914798122688428
62500000 7.0364272467783415
62600000 12.788562164028937


88700000 12.2818902950236
88800000 8.234593611125787
88900000 10.691819250802572
89000000 12.854753094063657
89100000 8.675233906494986
89200000 8.068953740201675
89300000 12.14737189837553
89400000 11.37172424684762
89500000 7.2093816158357376
89600000 12.185462766796542
89700000 11.891928125871907
89800000 12.525930269373056
89900000 13.088939417661708
90000000 13.430809073385594
90100000 13.178652525892216
90200000 13.347549229077776
90300000 13.274581729546382
90400000 12.841471681752482
90500000 12.831278070146155
90600000 12.242302421861226
90700000 12.032232363287774
90800000 12.025738828427453
90900000 5.043723769212887
91000000 11.993101931603833
91100000 11.938923419432427
91200000 4.075444818794282
91300000 12.121662085125704
91400000 11.829051452882672
91500000 5.72263170760596
91600000 12.431269905635274
91700000 11.79812359368814
91800000 11.820761769972496
91900000 12.506819943910108
92000000 12.717938147785281
92100000 13.213309404737284
92200000 13.022118450114503
9230

117800000 3.4004474719344384
117900000 4.2809174135628725
118000000 9.260491592608433
118100000 10.187551210146488
118200000 10.124439323024166
118300000 10.012804523223858
118400000 10.311117461922281
118500000 10.085073568668307
118600000 10.4431488581361
118700000 10.5871246301361
118800000 11.161096915340275
118900000 11.25771511759658
119000000 10.074601146438448
119100000 10.147600489632007
119200000 9.453749776432996
119300000 9.755136892343419
119400000 9.77255730544348
119500000 9.861440103426567
119600000 9.526217179845855
119700000 9.888105476947208
119800000 3.31868082051706
119900000 1.8655953966547887
120000000 7.042080945510145
120100000 9.362637715931623
120200000 9.390685850388337
120300000 9.674789376691212
120400000 9.657226699858475
120500000 9.743022488441078
120600000 9.295933937133944
120700000 9.97635890230618
120800000 9.975439185163662
120900000 11.728247080981042
121000000 11.228394600222158
121100000 10.731539603832436
121200000 10.564904021234144
121300000 

146700000 6.594814879025004
146800000 8.068440819329165
146900000 8.13260352488046
147000000 8.264787191553415
147100000 8.267072649201427
147200000 8.172634669596826
147300000 8.26937567474017
147400000 8.87235974975811
147500000 8.84257896215576
147600000 9.145016790233303
147700000 8.603306725173223
147800000 8.157461364401266
147900000 8.261339164774197
148000000 8.192195773115182
148100000 8.11469762047165
148200000 8.09793292093585
148300000 7.553319341046996
148400000 3.8616406709064126
148500000 6.483812618046582
148600000 5.345868774063043
148700000 3.472001404452417
148800000 7.712468780919983
148900000 7.864903105885163
149000000 8.005935995741416
149100000 8.015968987325254
149200000 7.953293032043451
149300000 7.968341915071697
149400000 8.36326616044322
149500000 8.882854209942947
149600000 8.773688001720434
149700000 8.7815689842726
149800000 8.059514379484886
149900000 7.940125187204897
150000000 8.098434442668704
150100000 8.082103062142561
150200000 8.0403397670864
15

176000000 6.778045156459927
176100000 6.852947637859002
176200000 6.698504743660299
176300000 6.5628331923531436
176400000 6.648806013831067
176500000 6.452918808348333
176600000 6.268801018359138
176700000 5.916649082679608
176800000 4.919521995251972
176900000 3.981168978878188
177000000 5.971742942491582
177100000 6.060569299057001
177200000 3.175761227150027
177300000 4.864069680717225
177400000 5.933778538192897
177500000 6.032163461098629
177600000 2.6444205055909094
177700000 6.001137710940399
177800000 5.988451266364301
177900000 6.537947323578521
178000000 6.370320951246317
178100000 6.672278129306512
178200000 6.6500234805287795
178300000 6.568706531694217
178400000 6.5261582264592475
178500000 6.452292151468486
178600000 6.504433148024165
178700000 6.073309742671777
178800000 5.893938131352996
178900000 2.806598329695742
179000000 5.972670777965832
179100000 5.997657696873671
179200000 5.723764200756181
179300000 2.5094579434007125
179400000 5.90320696696816
179500000 6.1682

205200000 4.56680106292954
205300000 2.144783583899961
205400000 4.264183341068067
205500000 4.269829126046792
205600000 4.0589164208368995
205700000 3.9997170903374615
205800000 3.9590264187678117
205900000 1.4720236771328814
206000000 4.192225866961193
206100000 4.022751107259037
206200000 3.9676670000660135
206300000 4.105503689500046
206400000 4.2730858755372525
206500000 1.8152842095287305
206600000 4.578567385833388
206700000 4.512486613619593
206800000 4.523858306820908
206900000 4.502897506541991
207000000 4.594923706070166
207100000 4.45842464210516
207200000 4.497382151825148
207300000 3.017305256873997
207400000 3.285571127892913
207500000 4.199815067062407
207600000 4.0711568441628465
207700000 3.8535813929332092
207800000 3.8498643580071645
207900000 3.141976308074436
208000000 2.467285458541524
208100000 3.8401037163216674
208200000 3.9084735734233735
208300000 4.058611610902828
208400000 4.162080525848216
208500000 3.8470667709558084
208600000 2.246721757233596
208700000

234000000 2.3287918033932824
234100000 2.177556279821315
234200000 2.2360953583803136
234300000 2.1196008763486294
234400000 2.0774700352449695
234500000 2.061061403139019
234600000 0.7319262877270593
234700000 2.005909485943728
234800000 2.0483013479405243
234900000 2.128069353549148
235000000 2.1572681599365136
235100000 2.1698713374869762
235200000 2.197459836246486
235300000 2.1818181261769034
235400000 0.9248851338882983
235500000 2.2206002394339417
235600000 2.2120878084994184
235700000 2.1374651084306397
235800000 0.9390462214376695
235900000 2.1335586378363636
236000000 2.1026107053900502
236100000 2.086461970623592
236200000 2.0100310019696375
236300000 2.072340749152225
236400000 2.009325445424338
236500000 1.9164215451427764
236600000 1.09482204114394
236700000 1.5617479772523482
236800000 1.9119926551809727
236900000 1.9571667970281683
237000000 2.0521526017115517
237100000 1.99865409446505
237200000 2.039561277762401
237300000 2.028913349519888
237400000 1.8949211312126206

262000000 0.0012256268865413135
262100000 0.0006833875883241495
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 5.969327267673281e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
